# HPXPy Benchmarks

This notebook benchmarks HPXPy array operations and compares them with NumPy.

**Topics covered:**
- Array creation performance
- Element-wise operations
- Reduction operations (sum, mean)
- Slicing and reshape overhead
- Scaling with array size

## 1. Setup

In [1]:
import time
import numpy as np
import hpxpy as hpx

# Initialize HPX
hpx.init()
print(f"HPXPy initialized with {hpx.num_threads()} threads")
print(f"NumPy version: {np.__version__}")

HPXPy initialized with 12 threads
NumPy version: 2.4.1


In [2]:
def benchmark(name, hpx_fn, np_fn, warmup=3, repeats=10):
    """Benchmark an operation and return timing results."""
    # Warmup
    for _ in range(warmup):
        hpx_fn()
        np_fn()
    
    # Time HPXPy
    hpx_times = []
    for _ in range(repeats):
        start = time.perf_counter()
        hpx_fn()
        hpx_times.append(time.perf_counter() - start)
    
    # Time NumPy
    np_times = []
    for _ in range(repeats):
        start = time.perf_counter()
        np_fn()
        np_times.append(time.perf_counter() - start)
    
    hpx_time = min(hpx_times) * 1000  # ms
    np_time = min(np_times) * 1000    # ms
    speedup = np_time / hpx_time if hpx_time > 0 else float('inf')
    
    return {
        'name': name,
        'hpxpy_ms': hpx_time,
        'numpy_ms': np_time,
        'speedup': speedup
    }

## 2. Array Creation Benchmarks

In [3]:
sizes = [10_000, 100_000, 1_000_000, 10_000_000]
creation_results = []

print(f"{'Operation':<20} {'Size':>12} {'HPXPy (ms)':>12} {'NumPy (ms)':>12} {'Speedup':>10}")
print("=" * 70)

for size in sizes:
    # arange
    result = benchmark(
        f"arange",
        lambda s=size: hpx.arange(s),
        lambda s=size: np.arange(s, dtype=np.float64)
    )
    result['size'] = size
    creation_results.append(result)
    print(f"{result['name']:<20} {size:>12,} {result['hpxpy_ms']:>12.3f} {result['numpy_ms']:>12.3f} {result['speedup']:>9.2f}x")
    
    # zeros
    result = benchmark(
        f"zeros",
        lambda s=size: hpx.zeros(s),
        lambda s=size: np.zeros(s, dtype=np.float64)
    )
    result['size'] = size
    creation_results.append(result)
    print(f"{result['name']:<20} {size:>12,} {result['hpxpy_ms']:>12.3f} {result['numpy_ms']:>12.3f} {result['speedup']:>9.2f}x")
    
    print("-" * 70)

Operation                    Size   HPXPy (ms)   NumPy (ms)    Speedup
arange                     10,000        0.004        0.004      0.96x
zeros                      10,000        0.004        0.002      0.51x
----------------------------------------------------------------------
arange                    100,000        0.023        0.032      1.36x
zeros                     100,000        0.015        0.008      0.52x
----------------------------------------------------------------------
arange                  1,000,000        0.211        0.307      1.46x
zeros                   1,000,000        0.153        0.077      0.50x
----------------------------------------------------------------------


arange                 10,000,000        2.121        3.067      1.45x
zeros                  10,000,000        1.248        0.623      0.50x
----------------------------------------------------------------------


## 3. Arithmetic Operations

In [4]:
arithmetic_results = []

print(f"{'Operation':<20} {'Size':>12} {'HPXPy (ms)':>12} {'NumPy (ms)':>12} {'Speedup':>10}")
print("=" * 70)

for size in sizes:
    np_arr = np.arange(size, dtype=np.float64)
    hpx_arr = hpx.arange(size)
    np_arr2 = np.ones(size, dtype=np.float64)
    hpx_arr2 = hpx.ones(size)
    
    # Scalar multiply
    result = benchmark(
        "multiply (a * 2)",
        lambda a=hpx_arr: a * 2,
        lambda a=np_arr: a * 2
    )
    result['size'] = size
    arithmetic_results.append(result)
    print(f"{result['name']:<20} {size:>12,} {result['hpxpy_ms']:>12.3f} {result['numpy_ms']:>12.3f} {result['speedup']:>9.2f}x")
    
    # Array add
    result = benchmark(
        "add (a + b)",
        lambda a=hpx_arr, b=hpx_arr2: a + b,
        lambda a=np_arr, b=np_arr2: a + b
    )
    result['size'] = size
    arithmetic_results.append(result)
    print(f"{result['name']:<20} {size:>12,} {result['hpxpy_ms']:>12.3f} {result['numpy_ms']:>12.3f} {result['speedup']:>9.2f}x")
    
    # Division
    result = benchmark(
        "divide (a / 2)",
        lambda a=hpx_arr: a / 2,
        lambda a=np_arr: a / 2
    )
    result['size'] = size
    arithmetic_results.append(result)
    print(f"{result['name']:<20} {size:>12,} {result['hpxpy_ms']:>12.3f} {result['numpy_ms']:>12.3f} {result['speedup']:>9.2f}x")
    
    print("-" * 70)

Operation                    Size   HPXPy (ms)   NumPy (ms)    Speedup
multiply (a * 2)           10,000        0.129        0.002      0.02x
add (a + b)                10,000        0.060        0.003      0.06x
divide (a / 2)             10,000        0.030        0.003      0.10x
----------------------------------------------------------------------
multiply (a * 2)          100,000        0.037        0.018      0.49x
add (a + b)               100,000        0.044        0.026      0.58x


divide (a / 2)            100,000        0.080        0.018      0.23x
----------------------------------------------------------------------
multiply (a * 2)        1,000,000        0.244        0.249      1.02x
add (a + b)             1,000,000        0.204        0.253      1.24x
divide (a / 2)          1,000,000        0.166        0.425      2.56x
----------------------------------------------------------------------


multiply (a * 2)       10,000,000        1.550        1.727      1.11x
add (a + b)            10,000,000        3.585        2.628      0.73x


divide (a / 2)         10,000,000        1.913        1.732      0.91x
----------------------------------------------------------------------


## 4. Reduction Operations

In [5]:
reduction_results = []

print(f"{'Operation':<20} {'Size':>12} {'HPXPy (ms)':>12} {'NumPy (ms)':>12} {'Speedup':>10}")
print("=" * 70)

for size in sizes:
    np_arr = np.arange(size, dtype=np.float64)
    hpx_arr = hpx.arange(size)
    
    # Sum
    result = benchmark(
        "sum",
        lambda a=hpx_arr: hpx.sum(a),
        lambda a=np_arr: np.sum(a)
    )
    result['size'] = size
    reduction_results.append(result)
    print(f"{result['name']:<20} {size:>12,} {result['hpxpy_ms']:>12.3f} {result['numpy_ms']:>12.3f} {result['speedup']:>9.2f}x")
    
    # Mean
    result = benchmark(
        "mean",
        lambda a=hpx_arr: hpx.mean(a),
        lambda a=np_arr: np.mean(a)
    )
    result['size'] = size
    reduction_results.append(result)
    print(f"{result['name']:<20} {size:>12,} {result['hpxpy_ms']:>12.3f} {result['numpy_ms']:>12.3f} {result['speedup']:>9.2f}x")
    
    # Min
    result = benchmark(
        "min",
        lambda a=hpx_arr: hpx.min(a),
        lambda a=np_arr: np.min(a)
    )
    result['size'] = size
    reduction_results.append(result)
    print(f"{result['name']:<20} {size:>12,} {result['hpxpy_ms']:>12.3f} {result['numpy_ms']:>12.3f} {result['speedup']:>9.2f}x")
    
    # Max
    result = benchmark(
        "max",
        lambda a=hpx_arr: hpx.max(a),
        lambda a=np_arr: np.max(a)
    )
    result['size'] = size
    reduction_results.append(result)
    print(f"{result['name']:<20} {size:>12,} {result['hpxpy_ms']:>12.3f} {result['numpy_ms']:>12.3f} {result['speedup']:>9.2f}x")
    
    print("-" * 70)

Operation                    Size   HPXPy (ms)   NumPy (ms)    Speedup
sum                        10,000        0.002        0.008      4.62x
mean                       10,000        0.004        0.010      2.72x
min                        10,000        0.009        0.006      0.66x
max                        10,000        0.009        0.006      0.64x
----------------------------------------------------------------------
sum                       100,000        0.024        0.044      1.84x
mean                      100,000        0.023        0.047      1.99x
min                       100,000        0.084        0.030      0.36x
max                       100,000        0.062        0.011      0.18x
----------------------------------------------------------------------
sum                     1,000,000        0.117        0.180      1.55x
mean                    1,000,000        0.117        0.181      1.55x
min                     1,000,000        0.614        0.101      0.16x
max   

mean                   10,000,000        1.323        2.006      1.52x


min                    10,000,000        6.148        1.338      0.22x


max                    10,000,000        6.368        1.717      0.27x
----------------------------------------------------------------------


## 5. Slicing and Reshape

In [6]:
slicing_results = []

print(f"{'Operation':<20} {'Size':>12} {'HPXPy (ms)':>12} {'NumPy (ms)':>12} {'Speedup':>10}")
print("=" * 70)

for size in sizes:
    np_arr = np.arange(size, dtype=np.float64)
    hpx_arr = hpx.arange(size)
    
    # Basic slice
    result = benchmark(
        "slice [:1000]",
        lambda a=hpx_arr: a[:1000],
        lambda a=np_arr: a[:1000]
    )
    result['size'] = size
    slicing_results.append(result)
    print(f"{result['name']:<20} {size:>12,} {result['hpxpy_ms']:>12.3f} {result['numpy_ms']:>12.3f} {result['speedup']:>9.2f}x")
    
    # Step slice
    result = benchmark(
        "slice [::2]",
        lambda a=hpx_arr: a[::2],
        lambda a=np_arr: a[::2]
    )
    result['size'] = size
    slicing_results.append(result)
    print(f"{result['name']:<20} {size:>12,} {result['hpxpy_ms']:>12.3f} {result['numpy_ms']:>12.3f} {result['speedup']:>9.2f}x")
    
    # Reshape
    sqrt_size = int(size ** 0.5)
    reshape_size = sqrt_size * sqrt_size
    np_reshape = np.arange(reshape_size, dtype=np.float64)
    hpx_reshape = hpx.arange(reshape_size)
    
    result = benchmark(
        "reshape (2D)",
        lambda a=hpx_reshape, s=sqrt_size: a.reshape((s, s)),
        lambda a=np_reshape, s=sqrt_size: a.reshape((s, s))
    )
    result['size'] = reshape_size
    slicing_results.append(result)
    print(f"{result['name']:<20} {reshape_size:>12,} {result['hpxpy_ms']:>12.3f} {result['numpy_ms']:>12.3f} {result['speedup']:>9.2f}x")
    
    # Flatten
    reshaped_np = np_reshape.reshape((sqrt_size, sqrt_size))
    reshaped_hpx = hpx_reshape.reshape((sqrt_size, sqrt_size))
    
    result = benchmark(
        "flatten",
        lambda a=reshaped_hpx: a.flatten(),
        lambda a=reshaped_np: a.flatten()
    )
    result['size'] = reshape_size
    slicing_results.append(result)
    print(f"{result['name']:<20} {reshape_size:>12,} {result['hpxpy_ms']:>12.3f} {result['numpy_ms']:>12.3f} {result['speedup']:>9.2f}x")
    
    print("-" * 70)

Operation                    Size   HPXPy (ms)   NumPy (ms)    Speedup
slice [:1000]              10,000        0.001        0.000      0.17x
slice [::2]                10,000        0.001        0.000      0.18x
reshape (2D)               10,000        0.001        0.000      0.31x
flatten                    10,000        0.003        0.002      0.72x
----------------------------------------------------------------------
slice [:1000]             100,000        0.001        0.000      0.18x
slice [::2]               100,000        0.001        0.000      0.18x
reshape (2D)               99,856        0.001        0.000      0.29x
flatten                    99,856        0.023        0.018      0.79x
----------------------------------------------------------------------
slice [:1000]           1,000,000        0.001        0.000      0.18x
slice [::2]             1,000,000        0.001        0.000      0.18x
reshape (2D)            1,000,000        0.001        0.000      0.31x
flatte

flatten                 9,998,244        2.092        1.469      0.70x
----------------------------------------------------------------------


## 6. Summary

In [7]:
all_results = creation_results + arithmetic_results + reduction_results + slicing_results

print("\n" + "=" * 70)
print("BENCHMARK SUMMARY")
print("=" * 70)

# Overall stats
speedups = [r['speedup'] for r in all_results]
faster_count = sum(1 for s in speedups if s > 1.0)
total_count = len(speedups)

print(f"\nTotal benchmarks: {total_count}")
print(f"HPXPy faster in: {faster_count}/{total_count} ({100*faster_count/total_count:.1f}%)")
print(f"Average speedup: {sum(speedups)/len(speedups):.2f}x")
print(f"Min speedup: {min(speedups):.2f}x")
print(f"Max speedup: {max(speedups):.2f}x")

# By category
print("\n--- By Category ---")
for name, results in [
    ("Creation", creation_results),
    ("Arithmetic", arithmetic_results),
    ("Reduction", reduction_results),
    ("Slicing", slicing_results)
]:
    speeds = [r['speedup'] for r in results]
    faster = sum(1 for s in speeds if s > 1.0)
    print(f"{name:12s}: avg {sum(speeds)/len(speeds):.2f}x, HPXPy faster in {faster}/{len(speeds)}")

# Best performing
print("\n--- Best HPXPy Performance ---")
sorted_results = sorted(all_results, key=lambda r: r['speedup'], reverse=True)[:5]
for r in sorted_results:
    print(f"{r['name']:20s} at size {r['size']:>12,}: {r['speedup']:.2f}x")

print(f"\nHPX threads used: {hpx.num_threads()}")


BENCHMARK SUMMARY

Total benchmarks: 52
HPXPy faster in: 15/52 (28.8%)
Average speedup: 0.80x
Min speedup: 0.02x
Max speedup: 4.62x

--- By Category ---
Creation    : avg 0.91x, HPXPy faster in 3/8
Arithmetic  : avg 0.75x, HPXPy faster in 4/12
Reduction   : avg 1.24x, HPXPy faster in 8/16
Slicing     : avg 0.33x, HPXPy faster in 0/16

--- Best HPXPy Performance ---
sum                  at size       10,000: 4.62x
mean                 at size       10,000: 2.72x
divide (a / 2)       at size    1,000,000: 2.56x
mean                 at size      100,000: 1.99x
sum                  at size      100,000: 1.84x

HPX threads used: 12


## 7. Visualization (Optional)

Requires matplotlib: `pip install matplotlib`

In [8]:
try:
    import matplotlib.pyplot as plt
    
    # Filter for largest size only
    large_results = [r for r in reduction_results if r['size'] == max(sizes)]
    
    if large_results:
        fig, ax = plt.subplots(figsize=(10, 6))
        
        names = [r['name'] for r in large_results]
        hpx_times = [r['hpxpy_ms'] for r in large_results]
        np_times = [r['numpy_ms'] for r in large_results]
        
        x = range(len(names))
        width = 0.35
        
        bars1 = ax.bar([i - width/2 for i in x], hpx_times, width, label='HPXPy')
        bars2 = ax.bar([i + width/2 for i in x], np_times, width, label='NumPy')
        
        ax.set_ylabel('Time (ms)')
        ax.set_title(f'Reduction Operations Performance (size={max(sizes):,})')
        ax.set_xticks(x)
        ax.set_xticklabels(names)
        ax.legend()
        
        plt.tight_layout()
        plt.show()
        
except ImportError:
    print("matplotlib not installed. Run: pip install matplotlib")

matplotlib not installed. Run: pip install matplotlib


## 8. Cleanup

In [9]:
hpx.finalize()
print("HPX runtime finalized")

HPX runtime finalized


## Notes

### When HPXPy is Faster

- **Large reductions**: `sum`, `mean`, `min`, `max` on arrays > 1M elements
- **Parallel transforms**: Operations that can use multiple threads
- **Multi-locality**: When distributed across multiple nodes

### When NumPy is Faster

- **Small arrays**: Thread overhead dominates for < 10K elements
- **Memory-bound ops**: Simple element-wise operations limited by memory bandwidth
- **Slicing**: Both are fast (views), minimal difference

### Tips for Best Performance

1. Use HPXPy for large arrays (> 1M elements)
2. Batch operations to amortize HPX overhead
3. Use multiple threads (`HPX_NUM_THREADS` environment variable)
4. Consider GPU backend for compute-intensive workloads